### Импорт библиотек

In [1]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import time
import os
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets

### Выгрузка данных
Cсылка на API: https://github.com/hhru/api 

In [2]:
text = f'SQL NAME:(\
"Junior Data Scientist" or "Data Scientist" or "Junior Data Science" or "Data Science"\
or "Джуниор Дата Саентист" or "Дата Cаентист"\
or "Machine Learning" or "Машинное обучение"\
or "Специалист по machine learning" or "Специалист по машинному обучению"\
or "Machine Learning Engineer" or "Machine Learning Developer" or "Machine Learning Researcher"\
or "Разработчик Machine Learning" or "Machine Learning инженер" \
or "ML Engineer" or "ML Developer" or "ML Researcher"\
or "Разработчик ML" or "ML инженер"\
or "Data Analyst" or "Analyst" or "Аналитик данных" or "Специалист по анализу данных" or "Аналитик"\
)'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=7)
date_to = '%s-%s-%s' % (dt_now.year, dt_now.month, dt_now.day)
date_from = '%s-%s-%s' % (dt_from.year, dt_from.month, dt_from.day)

url_0 = 'https://api.hh.ru/'
url_1 = url_0 + f'vacancies?text={text}&date_from={date_from}&date_to={date_to}&responses_count_enabled=True&per_page=100'

experiences = ['noExperience', 'between1And3']

In [3]:
request_data = pd.DataFrame()
for exp in experiences:
    page = 0
    url_2 = url_1 + f'&experience={exp}'
    try:
        r = requests.get(url_2 + f'&page={page}')
        data = json.loads(r.text)
        print(exp, ' - ', data['pages'], 'pages')
        items = data['items']
        for page in range(1, data['pages']):
            r = requests.get(url_2 + f'&page={page}')
            data = json.loads(r.text)
            items += data['items']
        df = pd.DataFrame(items)
        df['experience'] = exp
        request_data = pd.concat([request_data, df])
    except Exception as e:
        print(exp, '-', e)
        
request_data.reset_index(drop=True, inplace=True)
print('\nDataFrame shape:', request_data.shape)
pd.set_option('display.max_columns', None);
request_data.sample(3)

noExperience  -  3
between1And3  -  19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        2142 non-null   object
 1   premium                   2142 non-null   bool  
 2   name                      2142 non-null   object
 3   department                344 non-null    object
 4   has_test                  2142 non-null   bool  
 5   response_letter_required  2142 non-null   bool  
 6   area                      2142 non-null   object
 7   salary                    478 non-null    object
 8   type                      2142 non-null   object
 9   address                   1118 non-null   object
 10  response_url              0 non-null      object
 11  sort_point_distance       0 non-null      object
 12  published_at              2142 non-null   object
 13  created_at                2142 non-null

None

,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary,experience
1589,46193195,False,Продуктовый аналитик (Монетизация),"{'id': 'mailru-15478-other', 'name': 'VK'}",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Ленинградский пр...",None,None,2021-10-25T09:59:50+0300,2021-10-25T09:59:50+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/46193195?host=hh.ru,https://hh.ru/vacancy/46193195,[],"{'id': '15478', 'name': 'VK', 'url': 'https://...",{'requirement': 'Опыт проведения сплит-тестов ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 8},[],[],[],False,between1And3
2008,49078805,False,Системный аналитик фронт-офисной системы,"{'id': 'rosbank-599-bezo', 'name': '«РОСБАНК, ...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-22T16:38:56+0300,2021-10-22T16:38:56+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/49078805?host=hh.ru,https://hh.ru/vacancy/49078805,[],"{'id': '599', 'name': 'РОСБАНК, Societe Genera...",{'requirement': 'Понимание жизненного цикла <h...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 5},[],[],[],False,between1And3
862,49015492,False,Аналитик,None,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",None,"{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-22T19:12:31+0300,2021-10-22T19:12:31+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/49015492?host=hh.ru,https://hh.ru/vacancy/49015492,[],"{'id': '2460946', 'name': 'Самокат (ООО Умный ...",{'requirement': 'Опыт работы в сфере анализа <...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 34},[],[],[],False,between1And3


### Фильтрация датасета

In [65]:
# Извлечение нужных столбцов и информации из словарей в новый датафрейм

def return_id(x, key='id'):
    try:
        return x[key]
    except:
        return np.nan
    
vacancies = request_data.copy()[['id', 'name', 'published_at', 'url', 'alternate_url', 'experience']]

vacancies['employer'] = request_data['employer'].apply(return_id, key='name')
vacancies['city'] = request_data['area'].apply(return_id, key='name')
vacancies['type'] = request_data['type'].apply(return_id, key='name')
vacancies['schedule'] = request_data['schedule'].apply(return_id, key='name')
vacancies['responses'] = request_data['counters'].apply(return_id, key='responses')
vacancies['salary_from'] = request_data['salary'].apply(return_id, key='from')
vacancies['salary_to'] = request_data['salary'].apply(return_id, key='to')
vacancies['currency'] = request_data['salary'].apply(return_id, key='currency')

In [66]:
mask = (((vacancies['city']!='Москва') & 
         (vacancies['city']!='Нижний Новгород') & 
         (vacancies['schedule']!='Удаленная работа')) | 
         (vacancies['type']!='Открытая'))

vacancies.drop(labels=vacancies[mask].index, inplace=True)
vacancies.drop_duplicates(subset=['name', 'employer'], inplace=True)
vacancies.reset_index(drop=True, inplace=True)

print('DataFrame shape:', request_data.shape)
vacancies.sample(3)

DataFrame shape: (2142, 30)


,id,name,published_at,url,alternate_url,experience,employer,city,type,schedule,responses,salary_from,salary_to,currency
656,46235407,Бизнес-аналитик,2021-10-27T12:52:03+0300,https://api.hh.ru/vacancies/46235407?host=hh.ru,https://hh.ru/vacancy/46235407,between1And3,Forecsys,Москва,Открытая,Полный день,34,NaN,NaN,NaN
946,48242516,Системный аналитик (Капитал),2021-10-21T15:57:06+0300,https://api.hh.ru/vacancies/48242516?host=hh.ru,https://hh.ru/vacancy/48242516,between1And3,Компания БКС,Москва,Открытая,Удаленная работа,0,NaN,NaN,NaN
899,40982619,Системный аналитик (Медицина),2021-10-22T10:40:32+0300,https://api.hh.ru/vacancies/40982619?host=hh.ru,https://hh.ru/vacancy/40982619,between1And3,Сбербанк,Москва,Открытая,Полный день,3,NaN,NaN,NaN


### Просмотр содержимого

In [68]:
vacs = vacancies.head(5)['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [70]:
vacs

0    <IPython.core.display.HTML object>
1    <IPython.core.display.HTML object>
2    <IPython.core.display.HTML object>
3    <IPython.core.display.HTML object>
4    <IPython.core.display.HTML object>
Name: url, dtype: object

In [78]:
button = widgets.Button(description="show")
output = widgets.Output()

display(button, output)

counter = 0
def on_clicked(b):
    with output:
        try:
            global counter
            clear_output()
            print('№ вакансии: ', counter)
            display(HTML("<h3>" + str(vacancies['name'][counter]) + ' в ' + str(vacancies['employer'][counter]) + "</h3>"))
            display(vacs[counter])
            display(HTML('<a href="' + str(vacancies['alternate_url'][counter]) + '" target="_blank">Ссылка на вакансию</a>'))
            counter += 1
        except Exception as e:
            print('Вакансии закончились')
            
button.on_click(on_clicked)

Button(description='show', style=ButtonStyle())

Output()

In [51]:
#from bs4 import BeautifulSoup
#text = vacs_df['url'].apply(lambda x: BeautifulSoup(json.loads(requests.get(x).text)['description']).get_text())
#text[18]